# Inicialização

In [1]:
import tensorflow as tf

2022-05-31 08:58:07.808689: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-31 08:58:07.808775: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
print(tf.__version__)

2.7.0


# Conectando com servidor

In [3]:
import requests
import pandas as pd
import numpy as np

In [4]:
codigoGrupo = "Neg-100"
codigoPeriodo = "11"
codigoRegra = "t11s17v"

url = "http://vps-40d69db1.vps.ovh.ca:23005/api/ExemploTreinoAcaos/obtemConjuntoTreino?idPeriodo={}&codigoGrupoAcao={}&codigoRegraProjecao={}".format(codigoPeriodo,codigoGrupo,codigoRegra)

In [5]:
print(url)

http://vps-40d69db1.vps.ovh.ca:23005/api/ExemploTreinoAcaos/obtemConjuntoTreino?idPeriodo=11&codigoGrupoAcao=Neg-100&codigoRegraProjecao=t11s17v


In [6]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)

In [7]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [8]:
dfTratadoX = df.apply(trataLinhaX, axis='columns')

In [9]:
X = np.stack(dfTratadoX.values)

In [10]:
X.shape

(5280, 1800)

In [11]:
Y = np.stack(df['campoY'].values)

In [12]:
Y = Y.reshape(-1,1)

In [13]:
Y.shape

(5280, 1)

# Minha Rede Simples

In [14]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(1800,)),
    tf.keras.layers.Dense(1000, activation=tf.nn.relu),
    tf.keras.layers.Dense(500, activation=tf.nn.relu), 
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])


2022-05-31 09:02:35.269934: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-31 09:02:35.304764: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vps34755): /proc/driver/nvidia/version does not exist
2022-05-31 09:02:35.991627: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-31 09:02:39.765072: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 7200000 exceeds 10% of free system memory.
2022-05-31 09:02:40.001292: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 7200000 exceeds 10% of free system memory.
2022-05-31 09:02:40

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1000)              1801000   
                                                                 
 dense_1 (Dense)             (None, 500)               500500    
                                                                 
 dense_2 (Dense)             (None, 1)                 501       
                                                                 
Total params: 2,302,001
Trainable params: 2,302,001
Non-trainable params: 0
_________________________________________________________________


In [16]:
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
#model.compile(optimizer='adam' , loss='mean_squared_error')

In [18]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
                optimizer=RMSprop(learning_rate=1e-4),
                metrics=['accuracy'])

# Dados de Validação

In [19]:
urlValida = "http://vps-40d69db1.vps.ovh.ca:23005/api/ExemploTreinoAcaos/obtemConjuntoDesenvolvimento?idPeriodo={}&codigoGrupoAcao={}&codigoRegraProjecao={}".format(codigoPeriodo,codigoGrupo,codigoRegra)

In [20]:
print(urlValida)

http://vps-40d69db1.vps.ovh.ca:23005/api/ExemploTreinoAcaos/obtemConjuntoDesenvolvimento?idPeriodo=11&codigoGrupoAcao=Neg-100&codigoRegraProjecao=t11s17v


In [21]:
r = requests.get(urlValida)
j = r.json()
dfValida = pd.DataFrame.from_dict(j)

In [22]:
dfValida.head()

,campoX,campoY
0,"1.110304, 1.114927, 1.108322, 1.103038, 1.103...",0
1,"1.130732, 1.136893, 1.135524, 1.133470, 1.135...",0
2,"1.151473, 1.172511, 1.172511, 1.179523, 1.174...",0
3,"1.213515, 1.210640, 1.207045, 1.187635, 1.193...",0
4,"1.198813, 1.206973, 1.204006, 1.203264, 1.201...",0


In [23]:
dfTratadoValidaX = dfValida.apply(trataLinhaX, axis='columns')
XValida = np.stack(dfTratadoValidaX.values)
XValida.shape


(1936, 1800)

In [24]:
YValida = np.stack(dfValida['campoY'].values)


In [25]:
YValida = YValida.reshape(-1,1)

In [26]:
YValida.shape

(1936, 1)

In [ ]:
history = model.fit(x=X,y=Y, validation_data=(XValida, YValida) ,
                    steps_per_epoch=100,
                    epochs=15,
                    validation_steps=50,
                    verbose=2)

#EPOCHS = 20
#history = model.fit(
#      train_generator,
#      steps_per_epoch=100,  # 2000 images = batch_size * steps
#      epochs=EPOCHS,
#      validation_data=validation_generator,
#      validation_steps=50,  # 1000 images = batch_size * steps
#      verbose=2)

Epoch 1/15
100/100 - 97s - loss: 0.6148 - accuracy: 0.7394 - val_loss: 0.3056 - val_accuracy: 0.9143 - 97s/epoch - 970ms/step
Epoch 2/15


In [ ]:
# assign location
path='pesos/rede1.h5'
 
# save
model.save(path)